# 수행목표
- 사용자별 정밀도 및 재현율 계산 코드를 개발한다.
# 수행단계
- 아래 데이터를 활용해서 추천 개수에 따라 달라지는 정밀도와 재현율을 확인한다.

|사용자|가위손|나 홀로 집에|대부|사운드 오브 뮤직|시네마 천국|아마데우스|죽은 시인의 사회|터미네이터 2:오리지널|
|---|---|---|---|---|---|---|---|---|
|1917|1|0|1|1|1|1|1|1|
|10418|1|1|1|1|0|0|1|1|
|1980|1|1|1|1|1|1|0|0|
|2277|1|1|0|1|1|0|1|1|
|1805|0|0|1|1|1|1|1|1|
|5136|1|0|1|1|0|1|1|1|
|1561|1|0|1|0|1|1|1|1|
|1105|1|1|1|0|1|0|1|1|
|1312|1|1|0|0|1|0|1|1|
|3189|1|0|1|0|1|1|0|1|

- 위 데이터는 KMRD-small 데이터에서 사용자별 평가를 10개만 한 사용자와 영화별 평점 개수가 [60, 200)인 영화를 간추린 내용 중 일부이다.
- K개의 영화를 추천 했을 경우 사용자가 관심 있는 영화 비율을 Precision@K 라고 한다. (TP / (TP + FP))
- 사용자가 관심 있는 영화 중 추천한 영화 K개가 포함되는 비율을 Recall@K 라고 한다. (TP / (TP + FN))
- (가위손, 나 홀로 집에, 대부, 사운드 오브 뮤직) 4개 영화를 추천했다고 했을 경우 사용자별 정밀도와 재현율을 계산해본다.

In [1]:
# Library

import os
import sys
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

mpl.rcParams['font.family'] = 'AppleGothic'
mpl.rcParams['axes.unicode_minus'] = False

In [2]:
data = {
    "사용자": [1917, 10418, 1980, 2277, 1805, 5136, 1561, 1105, 1312, 3189],
    "가위손": [1, 1, 1, 1, 0, 1, 1, 1, 1, 1],
    "나 홀로 집에": [0, 1, 1, 1, 0, 0, 0, 1, 1, 0],
    "대부": [1, 1, 1, 0, 1, 1, 1, 1, 0, 1],
    "사운드 오브 뮤직": [1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
    "시네마 천국": [1, 0, 1, 1, 1, 0, 1, 1, 1, 1],
    "아마데우스": [1, 0, 1, 0, 1, 1, 1, 0, 0, 1],
    "죽은 시인의 사회": [1, 1, 0, 1, 1, 1, 1, 1, 1, 0],
    "터미네이터 2:오리지널": [1, 1, 0, 1, 1, 1, 1, 1, 1, 1],
}

df = pd.DataFrame(data)
df

,사용자,가위손,나 홀로 집에,대부,사운드 오브 뮤직,시네마 천국,아마데우스,죽은 시인의 사회,터미네이터 2:오리지널
0,1917,1,0,1,1,1,1,1,1
1,10418,1,1,1,1,0,0,1,1
2,1980,1,1,1,1,1,1,0,0
3,2277,1,1,0,1,1,0,1,1
4,1805,0,0,1,1,1,1,1,1
5,5136,1,0,1,1,0,1,1,1
6,1561,1,0,1,0,1,1,1,1
7,1105,1,1,1,0,1,0,1,1
8,1312,1,1,0,0,1,0,1,1
9,3189,1,0,1,0,1,1,0,1


In [3]:
# Precision@K
# 가위손, 나 홀로 집에, 대부, 사운드 오브 뮤직

def precision_at_k(df, movies, k):
    precisions = {}
    for user in df["사용자"]:
        user_data = df[df["사용자"] == user][movies]

        recommended_movies = user_data.iloc[0].sort_values(ascending=False).index[:k]
        relevant_movies = user_data.columns[user_data.iloc[0] == 1]

        precision = len(set(recommended_movies) & set(relevant_movies)) / len(recommended_movies)
        precisions[user] = precision

    return precisions

In [4]:
movies = ["가위손", "나 홀로 집에", "대부", "사운드 오브 뮤직"]

precision_results = precision_at_k(df, movies, k=len(movies))
precision_results

{1917: 0.75,
 10418: 1.0,
 1980: 1.0,
 2277: 0.75,
 1805: 0.5,
 5136: 0.75,
 1561: 0.5,
 1105: 0.75,
 1312: 0.5,
 3189: 0.5}

In [5]:
# Recall@K

def recall_at_k(df, movies, k):
    recalls = {}
    for user in df["사용자"]:
        user_data = df[df["사용자"] == user][movies]

        recommended_movies = user_data.iloc[0].sort_values(ascending=False).index[:k]
        relevant_movies = user_data.columns[user_data.iloc[0] == 1]

        recall = len(set(recommended_movies) & set(relevant_movies)) / len(relevant_movies)
        recalls[user] = recall

    return recalls


In [6]:
recall_results = recall_at_k(df, movies, k=len(movies))
recall_results

{1917: 1.0,
 10418: 1.0,
 1980: 1.0,
 2277: 1.0,
 1805: 1.0,
 5136: 1.0,
 1561: 1.0,
 1105: 1.0,
 1312: 1.0,
 3189: 1.0}